In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import  PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

examples = [
{
"question": "What do you know about France?",
"answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
"question": "What do you know about Italy?",
"answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
"question": "What do you know about Greece?",
"answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]

chat.predict("What do you know about France?")

France is a country located in Western Europe, known for its rich history, culture, and cuisine. It is the largest country in the European Union by land area and the third-largest in Europe overall. The capital city is Paris, which is famous for its iconic landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral.

France is a popular tourist destination, attracting millions of visitors each year to its beautiful countryside, historic cities, and world-renowned vineyards. The country is also known for its fashion industry, with Paris being considered one of the fashion capitals of the world.

French cuisine is highly regarded internationally, with dishes such as croissants, baguettes, cheese, and wine being staples of the French diet. The country is also known for its fine dining and Michelin-starred restaurants.

France has a rich cultural heritage, with famous artists such as Claude Monet, Vincent van Gogh, and Auguste Rodin hailing from the country. French literatu

'France is a country located in Western Europe, known for its rich history, culture, and cuisine. It is the largest country in the European Union by land area and the third-largest in Europe overall. The capital city is Paris, which is famous for its iconic landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral.\n\nFrance is a popular tourist destination, attracting millions of visitors each year to its beautiful countryside, historic cities, and world-renowned vineyards. The country is also known for its fashion industry, with Paris being considered one of the fashion capitals of the world.\n\nFrench cuisine is highly regarded internationally, with dishes such as croissants, baguettes, cheese, and wine being staples of the French diet. The country is also known for its fine dining and Michelin-starred restaurants.\n\nFrance has a rich cultural heritage, with famous artists such as Claude Monet, Vincent van Gogh, and Auguste Rodin hailing from the country. French l

In [ ]:
example_template="""
    Human: {question}
    AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

chain = prompt | chat

chain.invoke({
    'country': "Germany"
})


'\n    Human: What do you know about France?\n    AI: \nHere is what I know:\nCapital: Paris\nLanguage: French\nFood: Wine and Cheese\nCurrency: Euro\n\n\n\n\n    Human: What do you know about Italy?\n    AI: \nI know this:\nCapital: Rome\nLanguage: Italian\nFood: Pizza and Pasta\nCurrency: Euro\n\n\n\n\n    Human: What do you know about Greece?\n    AI: \nI know this:\nCapital: Athens\nLanguage: Greek\nFood: Souvlaki and Feta Cheese\nCurrency: Euro\n\n\n\nHuman: What do you know about Germany?'